# indicBERT Model

In [1]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

tokenizer = AutoTokenizer.from_pretrained("tokenizer_indicBERT/kaggle/working/tokenizer_indicBERT")
model_fine_tuned = AutoModelForTokenClassification.from_pretrained("my_indicBERT/kaggle/working/my_indicBERT")

In [5]:
with open('cs689_assignment.txt', 'r', encoding='utf-8') as file:
    # Read the contents of the file
    Inp1 = file.readlines()

corpus=[]

i=0
while (i<(len(Inp1)-1)):
    x = Inp1[i]
    x=x[2:-1]
    corpus.append(x)
    i+=3


In [8]:
# print(corpus)
print(len(corpus))

25


In [9]:
import torch

def get_ner(sentence):
    tok_sentence = tokenizer(sentence, return_tensors='pt')

    with torch.no_grad():
        logits = model_fine_tuned(**tok_sentence).logits.argmax(-1)
        predicted_tokens_classes = [
            model_fine_tuned.config.id2label[t.item()] for t in logits[0]]

        predicted_labels = []

        previous_token_id = 0
        word_ids = tok_sentence.word_ids()
        for word_index in range(len(word_ids)):
            if word_ids[word_index] == None:
                previous_token_id = word_ids[word_index]
            elif word_ids[word_index] == previous_token_id:
                previous_token_id = word_ids[word_index]
            else:
                predicted_labels.append(predicted_tokens_classes[word_index])
                previous_token_id = word_ids[word_index]

        ner_output = []
        l = min (len(sentence.split(' ')) , len(predicted_labels) )
        for index in range(l):
            ner_output.append(
                (sentence.split(' ')[index], predicted_labels[index]))
        return ner_output



In [10]:
# str = "दरअसल , जनवरी से चीन और नेपाल के सीमावर्ती क्षेत्रों को संचार सुविधा के लिए अब बैलून नेटवर्क सिस्टम की शुरूआत की जा रही है, जिसके साथ ही उत्तराखंड बैलून से नेटवर्क सुविधा देने वाला पहला राज्य बनेगा।  "
labeled_output = []
output=[]
for i in corpus:
    l = get_ner(i)
    labeled_output.append(l)
    temp=[]
    for j in l:
        temp.append(j[1])
    output.append(temp)



In [11]:
# tags_string = ' '.join(tags)

# Open a file in write mode
with open('tags_output_BERT.txt', 'w') as file:
    for tags in output:
        tags_string = ' '.join(tags)
        file.write(tags_string + '\n' )

# indicNER Model 

In [19]:
from transformers import AutoModelForTokenClassification, AutoConfig, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForTokenClassification, EarlyStoppingCallback, IntervalStrategy
import numpy as np

tokenizer_ner = AutoTokenizer.from_pretrained("tokenizer_indicNER/kaggle/working/tokenizer_indicNER")
model_fine_tuned_ner = AutoModelForTokenClassification.from_pretrained("my_indicNER/kaggle/working/my_indicNER")

In [20]:
import torch

def get_ner(sentence):
    tok_sentence = tokenizer_ner(sentence, return_tensors='pt')

    with torch.no_grad():
        logits = model_fine_tuned_ner(**tok_sentence).logits.argmax(-1)
        predicted_tokens_classes = [
            model_fine_tuned_ner.config.id2label[t.item()] for t in logits[0]]

        predicted_labels = []

        previous_token_id = 0
        word_ids = tok_sentence.word_ids()
        for word_index in range(len(word_ids)):
            if word_ids[word_index] == None:
                previous_token_id = word_ids[word_index]
            elif word_ids[word_index] == previous_token_id:
                previous_token_id = word_ids[word_index]
            else:
                predicted_labels.append(predicted_tokens_classes[word_index])
                previous_token_id = word_ids[word_index]

        ner_output = []
        l = min (len(sentence.split(' ')) , len(predicted_labels) )
        for index in range(l):
            ner_output.append(
                (sentence.split(' ')[index], predicted_labels[index]))
        return ner_output



In [21]:
labeled_output_ner = []
output_ner=[]
for i in corpus:
    l = get_ner(i)
    labeled_output_ner.append(l)
    temp=[]
    for j in l:
        temp.append(j[1])
    output_ner.append(temp)


In [22]:
# print(output_ner)

In [23]:
# tags_string = ' '.join(tags)

# Open a file in write mode
with open('tags_output_NER.txt', 'w') as file:
    for tags in output_ner:
        tags_string = ' '.join(tags)
        file.write(tags_string + '\n' )